In [ ]:
%matplotlib inline
from pylab import *

In [ ]:
from sklearn import datasets
olivetti = datasets.fetch_olivetti_faces()
X, y = olivetti.data, olivetti.target

In [ ]:
for i in range(10):
    face = olivetti.images[i]
    subplot(1, 10, i + 1)
    imshow(face.reshape((64, 64)), cmap='gray')
    axis('off')

In [ ]:
smiling_indices = [20,21,26,27,29, 40,41,42,43,44,45,46,\
                   47,48,49,50, 51, 52,61,69,70,72,75, 86,90,91,92, 93, 95,111,\
                   114,116,119,123,129,141,153,160,161,\
                   162,163,164,167,174,186,190,191,197,\
                   200,201,202,213,214,216,217, 219, 234,240,242,246,\
                   255,260,261,268,276,300,303, 305,310, 311,312, 313, 314, 315, 316,\
                   324,325, 326, 327, 328, 340,341,342, 343,\
                   348,349, 367,380,381,382,383,384,385,386,387,\
                   388,390, 391,393,396,397,399]

not_smiling_indices = list(set(range(400)) - set(smiling_indices))

In [ ]:
fig = plt.figure(figsize=(12, 12))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
for i in range(len(smiling_indices)):
    # plot the images in a matrix of 20x20
    p = fig.add_subplot(20, 20, i + 1)
    p.imshow(olivetti.images[smiling_indices[i]], cmap=plt.cm.bone)
    
    # label the image with the target value
    p.text(0, 30, "smiling")
    p.text(0, 60, str(smiling_indices[i]))
    p.axis('off')


In [ ]:

fig = plt.figure(figsize=(12, 12))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)
for i in range(len(not_smiling_indices)):
    # plot the images in a matrix of 20x20
    p = fig.add_subplot(20, 20, i + 1)
    p.imshow(olivetti.images[not_smiling_indices[i]], cmap=plt.cm.bone)

    # label the image with the target value
    p.text(0, 30, "not smiling")
    p.text(0, 60, str(not_smiling_indices[i]))
    p.axis('off')

In [ ]:
from sklearn.svm import SVC
svc_1 = SVC(kernel='linear', class_weight={1:3}, C=100)

data = olivetti.data

target = []
for i in range(400):
    if(i in smiling_indices):
        target.append(1)
    else:
        target.append(0)
        

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
        data, target, test_size=0.3, random_state=0)

In [ ]:
from sklearn.cross_validation import cross_val_score, KFold
from scipy.stats import sem

def evaluate_cross_validation(clf, X, y, K):
    # create a k-fold cross validation iterator
    cv = KFold(len(y), K, shuffle=True, random_state=0)
    # by default the score used is the one returned by score method of the estimator (accuracy)
    scores = cross_val_score(clf, X, y, cv=cv)
    print (scores)
    print ("Mean score: {0:.3f} (+/-{1:.3f})".format(
        np.mean(scores), sem(scores)))

In [ ]:
evaluate_cross_validation(svc_1, X_train, y_train, 5)

In [ ]:
from sklearn import metrics

def train_and_evaluate(clf, X_train, X_test, y_train, y_test):
    
    clf.fit(X_train, y_train)
    
    print ("Accuracy on training set:")
    print (clf.score(X_train, y_train))
    print ("Accuracy on testing set:")
    print (clf.score(X_test, y_test))
    
    y_pred = clf.predict(X_test)
    
    print ("Classification Report:")
    print (metrics.classification_report(y_test, y_pred))
    print ("Confusion Matrix:")
    print (metrics.confusion_matrix(y_test, y_pred))

In [ ]:
train_and_evaluate(svc_1, X_train, X_test, y_train, y_test)

In [ ]:
import cv2
from pylab import *
%matplotlib inline

face_cascade = cv2.CascadeClassifier('/home/francois/Projets/opencv-3.3.0/data/haarcascades/haarcascade_frontalface_default.xml')

img = cv2.imread('/home/francois/Projets/Smile Detect/smile_cora.jpg')
img = cv2.resize(img,None,fx=.4, fy=.4, interpolation = cv2.INTER_CUBIC)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
imshow(gray, cmap=plt.cm.bone)

In [ ]:
faces = face_cascade.detectMultiScale(gray, 1.3, 5)
for (x,y,w,h) in faces:
    roi_gray = gray[y:y+h, x:x+w]
    
imshow(roi_gray, cmap=plt.cm.bone)

In [ ]:
from scipy.ndimage import zoom
horizontal_offset = int(0.1 * w)
vertical_offset = int(0.06 * h)
extracted_face = gray[y+vertical_offset:y+h, 
                      x+horizontal_offset:x-horizontal_offset+w]
extracted_face = zoom(extracted_face, (64. / extracted_face.shape[0], 
                                 64. / extracted_face.shape[1]))

imshow(extracted_face, cmap=plt.cm.bone)

In [ ]:

new_extracted_face = zoom(extracted_face, (64. / extracted_face.shape[0], 
                                           64. / extracted_face.shape[1]))
new_extracted_face = new_extracted_face.astype(float32)
new_extracted_face /= float(new_extracted_face.max())
imshow(new_extracted_face, cmap=plt.cm.bone)

In [ ]:
svc_1.predict(new_extracted_face.ravel().reshape(1,-1))

In [ ]:
def detect_face(frame):
    cascPath = "/home/francois/Projets/opencv-3.3.0/data/haarcascades/haarcascade_frontalface_default.xml"
    faceCascade = cv2.CascadeClassifier(cascPath)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    detected_faces = faceCascade.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=6,
            minSize=(100, 100)
        )
    return gray, detected_faces

def extract_face_features(gray, detected_face, offset_coefficients):
    x, y, w, h = detected_face
    horizontal_offset = int(offset_coefficients[0] * w)
    vertical_offset = int(offset_coefficients[1] * h)
    extracted_face = gray[y+vertical_offset:y+h, 
                      x+horizontal_offset:x-horizontal_offset+w]
    new_extracted_face = zoom(extracted_face, (64. / extracted_face.shape[0], 
                                           64. / extracted_face.shape[1]))
    new_extracted_face = new_extracted_face.astype(float32)
    new_extracted_face /= float(new_extracted_face.max())
    return new_extracted_face

def predict_face_is_smiling(extracted_face):
    return svc_1.predict(extracted_face.ravel().reshape(1,-1))

In [ ]:
frame = cv2.imread('/home/francois/Projets/Smile Detect/smile_cora.jpg')

gray, detected_faces = detect_face(frame)

face_index = 0

for face in detected_faces:
        (x, y, w, h) = face
        if w > 100:
            # draw rectangle around face 
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
            # extract features
            extracted_face = extract_face_features(gray, face, (0.1, 0.06)) #(0.075, 0.05)

            # predict smile
            prediction_result = predict_face_is_smiling(extracted_face)

            # draw extracted face in the top right corner
            frame[face_index * 64: (face_index + 1) * 64, -65:-1, :] = cv2.cvtColor(extracted_face * 255, cv2.COLOR_GRAY2RGB)

            # annotate main image with a label
            if prediction_result == 1:
                cv2.putText(frame, "SMILING",(x,y), cv2.FONT_HERSHEY_SIMPLEX, 2, 155, 10)
            else:
                cv2.putText(frame, "not smiling",(x,y), cv2.FONT_HERSHEY_SIMPLEX, 2, 155, 10)

            # increment counter
            face_index += 1



imshow(frame, cmap=plt.cm.bone)